# Atelier 4

---

## Instructions

En se basant sur le fichier sql (ci-dessous) et la figure, vous devez réaliser les étapes suivantes en utilisant les requêtes SQL. </br>

![4_schema_figure.png](4_schema_figure.png)

---

### Partie 0 - Imports et fonctions

In [ ]:
import sqlite3 as sq
import mysql.connector as cnt

def connect_db(host, user, pswd, database = None):
    db = cnt.connect(
        host = host,
        user = user,
        passwd = pswd,
        database = database
    )
    return(db)

def exe(db, request, value = None, verbose = False, name = False):
    cursor = db.cursor(buffered = True)
    
    if value:
        cursor.execute(request, value)

    else :
        cursor.execute(request)

    db.commit()
    
    if verbose:
        result = cursor.fetchall()
        if name:
            result.insert(0, cursor.column_names)
        return(result)

---

### Partie 1 - création de la base et des tables :

Réaliser ces actions directement avec des requêtes SQL

#### 1.1/ créer une nouvelle base données nommée atelier4

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu')

query = 'CREATE DATABASE IF NOT EXISTS atelier4;'

exe(db, query)

#### 1.2/ Imiter les requêtes du fichier sql shema pour créer les tables et les remplir.

```

CREATE TABLE Movies (
  Code INTEGER PRIMARY KEY,
  Title VARCHAR(255) NOT NULL,
  Rating VARCHAR(255) 
);

CREATE TABLE MovieTheaters (
  Code INTEGER PRIMARY KEY,
  Name VARCHAR(255) NOT NULL,
  Movie INTEGER,  
    FOREIGN KEY (Movie) REFERENCES Movies(Code)
) ENGINE=INNODB;

INSERT INTO Movies(Code,Title,Rating) VALUES(1,'Citizen Kane','PG');
INSERT INTO Movies(Code,Title,Rating) VALUES(2,'Singin'' in the Rain','G');
INSERT INTO Movies(Code,Title,Rating) VALUES(3,'The Wizard of Oz','G');
INSERT INTO Movies(Code,Title,Rating) VALUES(4,'The Quiet Man',NULL);
INSERT INTO Movies(Code,Title,Rating) VALUES(5,'North by Northwest',NULL);
INSERT INTO Movies(Code,Title,Rating) VALUES(6,'The Last Tango in Paris','NC-17');
INSERT INTO Movies(Code,Title,Rating) VALUES(7,'Some Like it Hot','PG-13');
INSERT INTO Movies(Code,Title,Rating) VALUES(8,'A Night at the Opera',NULL);
 
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(1,'Odeon',5);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(2,'Imperial',1);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(3,'Majestic',NULL);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(4,'Royale',6);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(5,'Paraiso',3);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(6,'Nickelodeon',NULL);

```

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'atelier4')

queries = ["""
    CREATE TABLE IF NOT EXISTS Movies (
        Code INTEGER PRIMARY KEY,
        Title VARCHAR(255) NOT NULL,
        Rating VARCHAR(255) 
    )ENGINE = InnoDB;
"""]

queries.append("""
    CREATE TABLE IF NOT EXISTS MovieTheaters (
        Code INTEGER PRIMARY KEY,
        Name VARCHAR(255) NOT NULL,
        Movie INTEGER,  
        FOREIGN KEY (Movie) REFERENCES Movies(Code)
    ) ENGINE = InnoDB;
""")

queries.append("""
    INSERT INTO Movies VALUES
        (1,'Citizen Kane','PG'),
        (2,'Singin'' in the Rain','G'),
        (3,'The Wizard of Oz','G'),
        (4,'The Quiet Man',NULL),
        (5,'North by Northwest',NULL),
        (6,'The Last Tango in Paris','NC-17'),
        (7,'Some Like it Hot','PG-13'),
        (8,'A Night at the Opera',NULL);
""")

queries.append("""
    INSERT INTO MovieTheaters 
        VALUES(1,'Odeon',5),
        (2,'Imperial',1),
        (3,'Majestic',NULL),
        (4,'Royale',6),
        (5,'Paraiso',3),
        (6,'Nickelodeon',NULL);
""")

for query in queries:
    exe(db, query)

---

### Partie 2 - réaliser une sélection de données :

#### 2.1/ Sélectionnez le titre de tous les films.

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier4")

query = """
    SELECT Title
    FROM Movies;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response[0]}")

#### 2.2/ Afficher toutes les classifications distinctes dans la base de données (utilsier DISTINCT).

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier4")

query = """
    SELECT DISTINCT Rating
    FROM Movies
    WHERE Rating IS NOT NULL;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response[0]}")

#### 2.3/ Afficher tous les films non classés (si un film est non classé, sa valeur est NULL).

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier4")

query = """
    SELECT Title
    FROM Movies
    WHERE Rating IS NULL;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response[0]}")

#### 2.4/ Sélectionner tous les cinémas qui ne diffusent pas de film actuellement (utiliser NULL également).

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier4")

query = """
    SELECT Name
    FROM MovieTheaters
    WHERE Movie IS NULL;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response[0]}")

---

### Partie 3 - utiliser les jointures :

#### 3.1/ Sélectionner toutes les données de tous les cinémas ainsi que les données du film qui est projeté en salle (si un film est projeté).

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier4")

query = """
    SELECT *
    FROM MovieTheaters
    INNER JOIN Movies ON Movies.Code = MovieTheaters.Movie
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response}")

#### 3.2/ Ajouter le film non coté "One, Two, three".

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'atelier4')

query = """
    INSERT INTO Movies VALUES (9, 'One, Two, three', NULL);
"""

responses = exe(db, query)

#### 3.3/ Fixer la classification de tous les films non classés à "G".

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'atelier4')

query = """
    UPDATE Movies
    SET Rating = 'G'
    WHERE Rating IS NULL;
"""

responses = exe(db, query)

---

#### Partie 4 - BONUS :

#### 4.1/ Sélectionnez toutes les données de tous les films et, si ce film est diffusé dans une salle de cinéma, affichez les données de la salle de cinéma (Utiliser LEFT JOIN : https://sql.sh/cours/jointures/left-join)

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier4")

query = """
    SELECT *
    FROM Movies
    LEFT JOIN MovieTheaters ON Movies.Code = MovieTheaters.Movie;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response}")

#### 4.2 Afficher les titres des films qui ne sont actuellement diffusés dans aucun cinéma.

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier4")

query = """
    SELECT Title
    FROM Movies
    LEFT JOIN MovieTheaters ON Movies.Code = MovieTheaters.Movie
    WHERE MovieTheaters.Movie IS NULL;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response[0]}")

#### 4.3 Supprimer les salles de cinéma qui projettent des films classés "NC-17".

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'atelier4')

query = """
    DELETE MovieTheaters
    FROM MovieTheaters
    INNER JOIN Movies ON Movies.Code = MovieTheaters.Movie
    WHERE  Rating = 'NC-17';
"""

responses = exe(db, query)